In [5]:
import pandas as pd
import numpy as np
#import requests
import re

In [4]:
#считываем файл и добавляем марки машин, взятые с borauto.ru и с интенета
df = pd.read_csv('toretto-car.csv')

with open('marks1.txt') as f:
    marks1 = f.readlines()

with open('marks2.txt') as f:
    marks2 = f.readlines()

marks1 = list(map(lambda x: x.strip(), marks1))
marks1 = list(map(lambda x: x.lower(), marks1))
marks2 = list(map(lambda x: x.strip(), marks2))
marks2 = list(map(lambda x: x.lower(), marks2))
marks = set(marks1).union(set(marks2))
marks1[marks1.index("заз")] = 'zaz'
marks1[marks1.index("газ")] = 'gaz'
marks1[marks1.index("тагаз")] = 'tagaz'

In [6]:
# добавялем столбецы с марками автомобилей, взятые из полей ym_pv_URL и ym_pv_referer соответственно
df['marks'] = df['ym_pv_URL'].str.extract(f"({'|'.join(marks)})")
df['refer_marks'] = df['ym_pv_referer'].str.extract(f"({'|'.join(marks)})")

In [ ]:
df['special_offer'] = df['ym_pv_URL'].str.contains('special')

In [7]:
# вспомогательные функции для добавления моделей машин
def func1(s,mark):
    res = re.search(  f"(?<={mark}/)(.*)(?=/)", s)
    if res: return res[0]
    return res

def func2(s,mark):
    res = re.search(  f'(?<={mark}/)(.*)(?=\?)', s)
    if res: return res[0]
    return res

def f(s,mark):
    if pd.isna(s): return None
    if func1(s,mark):
        return func1(s,mark)
    return func2(s,mark)

In [8]:
# добавялем столбцы с моделями автомобилей, взятые из полей ym_pv_URL и ym_pv_referer соответственно
df['model'] = df.apply(lambda x: f(x['ym_pv_URL'], x['marks']), axis=1)
df['refer_model'] = df.apply(lambda x: f(x['ym_pv_referer'], x['refer_marks']), axis=1)

In [9]:
# вспомогательная функция для нахождения уникальных номеров машин
def g(s,mark,model):
    if pd.isna(s): return None
    res = re.search(  f"(?<={mark}/{model}/)([nu]\d*)", s)
    if res: return res[0]
    return res

In [10]:
#находим ид машин, взятые из полей ym_pv_URL и ym_pv_referer соответственно
df['car_id'] = df.apply(lambda x: g(x['ym_pv_URL'], x['marks'], x['model']), axis=1)
df['car_id_refer'] = df.apply(lambda x: g(x['ym_pv_referer'], x['refer_marks'], x['refer_model']), axis=1)

In [11]:
#находим все ид машин
car_id = df['car_id'].unique()
car_id_refer = df['car_id_refer'].unique()
car_id = car_id[1:]
car_id_refer = car_id_refer[1:]
cars_id = set(car_id).union(set(car_id_refer))

In [12]:
#сохраняем словарь с ид машины и соответсующей маркой и моделью
d = {}
for id in cars_id:
    if id in car_id:
        row = df[df['car_id'] == id].iloc[0]
        mark = df[df['car_id'] == id].iloc[0].marks
        model = df[df['car_id'] == id].iloc[0].model
    else:
        row = df[df['car_id_refer'] == id].iloc[0]
        mark = df[df['car_id_refer'] == id].iloc[0].refer_marks
        model = df[df['car_id_refer'] == id].iloc[0].refer_model
    d[id] = [mark, model]

In [13]:
#добавим ид из столбца ym_pv_URL, которых раньше не было
def func_car_id(s):
    for id in cars_id:
        res = re.search(  f"{id}", s)
        if res: return res[0]
    return None
df['car_id'] = df.apply(lambda x: x['car_id'] if x['car_id'] else func_car_id(x['ym_pv_URL']), axis=1)

In [14]:
#дополним стороки маркой и моделью, в которых в ссылке (столбец ym_pv_URL) есть только ид машины
def func_mark(s,car_id):
    res = re.search(  f"{car_id}", s)
    if res: return d[res[0]][0]
    return res

def func_model(s,car_id):
    res = re.search(  f"{car_id}", s)
    if res: return d[res[0]][1]
    return res

df['marks'] = df.apply(lambda x: x['marks'] if  not pd.isna(x['marks']) else func_mark(x['ym_pv_URL'], x['car_id']), axis=1)
df['model'] = df.apply(lambda x: x['model'] if x['model'] else func_model(x['ym_pv_URL'], x['car_id']), axis=1)

In [15]:
#список всех извсетных моделей
model_list = df['model'].unique()
model_list = model_list[1:]


In [16]:

def f_model(s):
    for model in model_list:
        res = re.search(  f"{model}", s)
        if res: return res[0]
    return None

df['model'] = df.apply(lambda x : x['model'] if x['model'] else f_model(x['ym_pv_URL']), axis=1)

In [17]:
#удаляем вспомогательные столбцы
new_df = df.drop(columns=['refer_marks', 'refer_model', 'car_id_refer'])
new_df = new_df.rename(columns={"marks": "mark"})

In [18]:
# добавим ид машин из ссылок, в которых есть только ид машины (без марки и модели)
def g_car_id(s):
    res = re.search(  f"(?<=#car-)([nu]\d*)", s)
    if res: return res[0]
    return res

new_df['car_id'] = df.apply(lambda x : x['car_id'] if x['car_id'] else g_car_id(x['ym_pv_URL']), axis=1)

In [ ]:
#сохраняем новую таблицу, с которой продолжим работать
new_df.to_csv('toretto_car_new.csv', index=False)